In [16]:
#Dependencies and file import
import pandas as pd
pd.options.mode.chained_assignment = None
school_data= "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"


#Read data in with Pandas
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

school_df=school_df.rename(columns={'School ID':'School ID','school_name': 'School Name', 'type': 'School Type', 'size':'Total Students', 'budget': 'Budget'})
school_df.count()

School ID         15
School Name       15
School Type       15
Total Students    15
Budget            15
dtype: int64

In [15]:
student_df=student_df.rename(columns={'Student ID':'Student ID', 'student_name':'Student Name', 'gender':'Gender', 'grade':'Grade', 'school_name': 'School Name',
       'reading_score':'Reading Score', 'math_score':'Math Score'})
student_df.count()

Student ID       39170
Student Name     39170
Gender           39170
Grade            39170
School Name      39170
Reading Score    39170
Math Score       39170
dtype: int64

In [3]:
#Merge student_df and school_df
complete_school_df = pd.merge(student_df, school_df, how="left", on=["School Name", "School Name"])
complete_school_df.columns

Index(['Student ID', 'Student Name', 'Gender', 'Grade', 'School Name',
       'Reading Score', 'Math Score', 'School ID', 'School Type',
       'Total Students', 'Budget'],
      dtype='object')

In [4]:
# District Summary

#Total schools
total_schools = len(complete_school_df["School Name"].unique())
#Total students
total_students = complete_school_df["Student Name"].count() 
#Total budget
total_budget = school_df["Budget"].sum()         
#Average math score
avg_math = complete_school_df["Math Score"].mean()
                    
#Average reading score
avg_reading = complete_school_df["Reading Score"].mean()
                    
#% passing math (the percentage of students who passed math)
passing_math = (complete_school_df.loc[complete_school_df["Math Score"] >= 70,"Math Score"].count() /\
                                   total_students)*100
                    
#% passing reading (the percentage of students who passed reading)
passing_reading = (complete_school_df.loc[complete_school_df["Reading Score"] >= 70,"Reading Score"].count() /\
                                   total_students)*100
                    
#% overall passing (the percentage of students who passed math AND reading)
overall_passing = (complete_school_df[(complete_school_df["Math Score"] >= 70) \
                    & (complete_school_df["Reading Score"] >= 70)]["Student Name"].count() \
                        /total_students)*100

#District Summary DataFrome in a list of dictionaries
district_summary_df = pd.DataFrame([{"Total Schools": total_schools, 
                                    "Total Students": total_students, 
                                   "Total Budget": total_budget, 
                                   "Average Math Score": avg_math,
                                   "Average Reading Score": avg_reading, 
                                   "% Passing Math": passing_math, 
                                    "% Passing Reading": passing_reading,
                                   "% Overall Passing": overall_passing}])

#Formatting
district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,.0f}".format)
district_summary_df["Total Budget"]=district_summary_df["Total Budget"].map("${:,.0f}".format)
district_summary_df["Average Math Score"]=district_summary_df["Average Math Score"].map("{:,.2f}".format)
district_summary_df["Average Reading Score"]=district_summary_df["Average Reading Score"].map("{:,.2f}".format)
district_summary_df["% Passing Math"]=district_summary_df["% Passing Math"].map("{:,.2f}".format)
district_summary_df["% Passing Reading"]=district_summary_df["% Passing Reading"].map("{:,.2f}".format)
district_summary_df["% Overall Passing"]=district_summary_df["% Overall Passing"].map("{:,.2f}".format)


district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,65.17


In [5]:
# School Summary
#Create an overview table that summarizes key metrics about each school, including:
#School Name
#School Type
#Total Students
#Total School Budget
#Per Student Budget
#Average Math Score
#Average Reading Score

#% Passing Math
complete_school_df["math_passing"]=complete_school_df["Math Score"]>=70

#% Passing Reading
complete_school_df["reading_passing"]=complete_school_df["Reading Score"]>=70

#% Overall Passing (The percentage of students that passed math and reading.)
complete_school_df["overall_passing"]=(complete_school_df["Math Score"] >= 70) & (complete_school_df["Reading Score"] >= 70)                   

In [6]:
#Set School Name as index and sort
sorted_complete_school_df = complete_school_df.set_index("School Name").sort_values("School Name")

#Group by School calculating means
grouped_school_df = sorted_complete_school_df.groupby(["School Name"]).mean()

#Merge grouped dataframe with original school data frame
school_data_sum = pd.merge(grouped_school_df,school_df,how="left",on=["School Name","School Name"])
school_data_sum.columns

Index(['School Name', 'Student ID', 'Reading Score', 'Math Score',
       'School ID_x', 'Total Students_x', 'Budget_x', 'math_passing',
       'reading_passing', 'overall_passing', 'School ID_y', 'School Type',
       'Total Students_y', 'Budget_y'],
      dtype='object')

In [7]:
#     Create a dataframe to hold the above results
school_summary_df = school_data_sum.set_index("School Name")
school_summary_df = school_summary_df.rename(columns={'Reading Score':'Average Reading Score', 'Math Score':'Average Math Score', 
                                                      'Total Students_x':'Total Students', 'Budget_x':'Total School Budget',
                                                      'math_passing':"% Passing Math",'reading_passing':"% Passing Reading",
                                                      'overall_passing':'% Overall Passing'})
school_summary_df["Per Student Budget"] = school_summary_df["Total School Budget"]/school_summary_df["Total Students"]
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"]*100
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"]*100
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"]*100
school_summary_df = school_summary_df[["School Type","Total Students","Total School Budget","Per Student Budget",
                                       "Average Math Score","Average Reading Score","% Passing Math",
                                       "% Passing Reading",'% Overall Passing']]
#Formatting
school_summary_df["Total Students"]=school_summary_df["Total Students"].map("{:,.0f}".format)
school_summary_df["Total School Budget"]=school_summary_df["Total School Budget"].map("${:,.0f}".format)
school_summary_df["Per Student Budget"]=school_summary_df["Per Student Budget"].map("${:,.0f}".format)
school_summary_df["Average Math Score"]=school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"]=school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["% Passing Math"]=school_summary_df["% Passing Math"].map("{:,.2f}".format)
school_summary_df["% Passing Reading"]=school_summary_df["% Passing Reading"].map("{:,.2f}".format)
school_summary_df["% Overall Passing"]=school_summary_df["% Overall Passing"].map("{:,.2f}".format)
school_summary_df.index.name = None

school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


In [8]:
#Top Performing Schools (By % Overall Passing)
#Sort and display the top five performing schools by % overall passing.

top_schools = school_summary_df.sort_values(["% Overall Passing"],ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


In [9]:
#Bottom Performing Schools (By % Overall Passing)

#Sort and display the five worst-performing schools by % overall passing.

bottom_schools=school_summary_df.sort_values(["% Overall Passing"])
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54


In [10]:
#Math Scores by Grade

#Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

#Create a pandas series for each grade. Hint: use a conditional statement.

#Group each series by school

#Combine the series into a dataframe

#Optional: give the displayed data cleaner formatting

#Creating Series on grade, then grouping by School Name to average Math Score
math_ninth = complete_school_df[complete_school_df['Grade'] == "9th"].groupby(['School Name']).mean()["Math Score"]
math_tenth = complete_school_df[complete_school_df['Grade'] == "10th"].groupby(['School Name']).mean()["Math Score"]
math_eleventh = complete_school_df[complete_school_df['Grade'] == "11th"].groupby(['School Name']).mean()["Math Score"]
math_twelfth = complete_school_df[complete_school_df['Grade'] == "12th"].groupby(['School Name']).mean()["Math Score"]


#Creating DataFrame
math_by_grade = pd.DataFrame({"9th": math_ninth, "10th": math_tenth,"11th": math_eleventh,"12th":math_twelfth})

#Formatting
math_by_grade["9th"]=math_by_grade["9th"].map("{:.2f}".format)
math_by_grade["10th"]=math_by_grade["10th"].map("{:.2f}".format)
math_by_grade["11th"]=math_by_grade["11th"].map("{:.2f}".format)
math_by_grade["12th"]=math_by_grade["12th"].map("{:.2f}".format)
math_by_grade.index.name = None
math_by_grade

                                   

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


In [11]:
#  Reading Score by Grade
# Perform the same operations as above for reading scores


#Creating Series on grade, then grouping by School Name to average Reading Score
reading_ninth = complete_school_df[complete_school_df['Grade'] == "9th"].groupby(['School Name']).mean()["Reading Score"]
reading_tenth = complete_school_df[complete_school_df['Grade'] == "10th"].groupby(['School Name']).mean()["Reading Score"]
reading_eleventh = complete_school_df[complete_school_df['Grade'] == "11th"].groupby(['School Name']).mean()["Reading Score"]
reading_twelfth = complete_school_df[complete_school_df['Grade'] == "12th"].groupby(['School Name']).mean()["Reading Score"]


#Creating DataFrame
reading_by_grade = pd.DataFrame({"9th": reading_ninth, "10th": reading_tenth,
                                 "11th": reading_eleventh,"12th":reading_twelfth})
#Formatting
reading_by_grade["9th"]=reading_by_grade["9th"].map("{:.2f}".format)
reading_by_grade["10th"]=reading_by_grade["10th"].map("{:.2f}".format)
reading_by_grade["11th"]=reading_by_grade["11th"].map("{:.2f}".format)
reading_by_grade["12th"]=reading_by_grade["12th"].map("{:.2f}".format)
reading_by_grade.index.name = None
reading_by_grade


,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


In [12]:
#Scores by School Spending
 
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#Use 4 reasonable bins to group school spending. Include in the table each of the following:
#Average Math Score
#Average Reading Score
#% Passing Math
#% Passing Reading
#Overall Passing Rate (Average of the above two)


#Bins and Bin names
spending_bins = [0,585,630,645,680]
spending_bins_names = ["<$585","$585-630","$630-645","$645-680"]

#Will use school_summary_df to create the binned DataFrames, but need to convert strings to numbers/float
conv_sch_sum_df = school_summary_df
conv_sch_sum_df['Per Student Budget'] = conv_sch_sum_df['Per Student Budget'].str.replace("$", "", regex=True).astype('float')
conv_sch_sum_df['Total Students'] = conv_sch_sum_df['Total Students'].str.replace(",", "").astype('float')
conv_sch_sum_df=conv_sch_sum_df.astype({"Total Students":float,"Average Math Score":float,
                                        "Average Reading Score":float,"% Passing Math":float,"% Passing Reading":float,
                                        "% Overall Passing":float})                                                                                                          

spending_df = conv_sch_sum_df[['Per Student Budget','Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing']]


#Binning
spending_df["Spending Ranges (Per Student)"] = pd.cut(spending_df["Per Student Budget"],
                                                      spending_bins,labels=spending_bins_names,include_lowest=True)
#Group by and average within the bin
spending_df=spending_df.groupby("Spending Ranges (Per Student)").mean()

#Delete Per Student Budget column
spending_df=spending_df.drop("Per Student Budget",axis=1)

#Formatting
spending_df["Average Math Score"]=spending_df["Average Math Score"].map("{:.2f}".format)
spending_df["Average Reading Score"]=spending_df["Average Reading Score"].map("{:.2f}".format)
spending_df["% Passing Math"]=spending_df["% Passing Math"].map("{:.2f}".format)
spending_df["% Passing Reading"]=spending_df["% Passing Reading"].map("{:.2f}".format)
spending_df["% Overall Passing"]=spending_df["% Overall Passing"].map("{:.2f}".format)


spending_df




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [13]:
#Scores by School Size
 
#Perform the same operations as above, based on school size.


#Bins and Bin names
size_bins = [0,1000,2000,5000]
size_bins_names = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]


size_df = conv_sch_sum_df[['Total Students','Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing']]


#Binning
size_df["School Size"] = pd.cut(size_df["Total Students"],
                                                      size_bins,labels=size_bins_names,include_lowest=True)
#Group by and average within the bin
size_df=size_df.groupby("School Size").mean()

#Delete Per Student Budget column
size_df=size_df.drop("Total Students",axis=1)

#Formatting
size_df["Average Math Score"]=size_df["Average Math Score"].map("{:.2f}".format)
size_df["Average Reading Score"]=size_df["Average Reading Score"].map("{:.2f}".format)
size_df["% Passing Math"]=size_df["% Passing Math"].map("{:.2f}".format)
size_df["% Passing Reading"]=size_df["% Passing Reading"].map("{:.2f}".format)
size_df["% Overall Passing"]=size_df["% Overall Passing"].map("{:.2f}".format)


size_df



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.89
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.28


In [14]:
#Scores by School Type
 
#Perform the same operations as above, based on school type


type_df = conv_sch_sum_df.groupby("School Type").mean()
type_df=type_df.drop(["Total Students","Per Student Budget"],axis=1)

#Formatting
type_df["Average Math Score"]=type_df["Average Math Score"].map("{:.2f}".format)
type_df["Average Reading Score"]=type_df["Average Reading Score"].map("{:.2f}".format)
type_df["% Passing Math"]=type_df["% Passing Math"].map("{:.2f}".format)
type_df["% Passing Reading"]=type_df["% Passing Reading"].map("{:.2f}".format)
type_df["% Overall Passing"]=type_df["% Overall Passing"].map("{:.2f}".format)


type_df



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


In [ ]:
# # Observable Conclusions
# Based on these tables, we can observe a couple trends. Charter schools perform better than District schools. 
# The five best-performing schools are charter schools, while the five worst-performing schools are district schools. 
# Larger schools (2000-5000 students) have lower averages and less passing students than Small or Medium schools, 
# which makes sense since larger schools have a bigger student to teacher ratio. It also appears that math and reading scores 
# are higher when spending per student is lower. I believe that conincides with school size. Smaller schools have a smaller budget, 
# but teachers have less students, therefore teachers can afford to spend more time with individual students.